# Exploratory Data Analysis Part 1: Data Selection

In [14]:
# Libraries importing
import numpy as np
import pandas as pd

In [15]:
# Get training data from .txt file
data = pd.read_csv('../data/PAKDD2010_Modeling_Data.txt',
                   header=None,
                   delimiter='\t',
                   encoding='ISO-8859-1',
                   index_col=0
                   )

/tmp/ipykernel_1043/1835940254.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/PAKDD2010_Modeling_Data.txt',


In [16]:
# Get name and description of data columns
var_names = pd.read_excel('../data/PAKDD2010_VariablesList.XLS')

In [17]:
# Analyze if there are duplicated variable names
var_names['Var_Title'].duplicated().value_counts()

False    53
True      1
Name: Var_Title, dtype: int64

In [18]:
# As there are a variable with the same name that other, let's rename the
# second.
cols = []
for column in var_names['Var_Title'][1:]:     # Isolate ID_CLIENT because it is
                                              # used as dataframe index
    if column in cols:
        count = cols.count(column) + 1
        cols.append(f'{column}_{count}')
    else:
        cols.append(column)

In [19]:
# Set Var_Title as Columns names for or training dataset
data.columns = cols

In [20]:
data.shape[0]

50000

In [21]:
# Count Null values per column
data.isna().sum()

CLERK_TYPE                            0
PAYMENT_DAY                           0
APPLICATION_SUBMISSION_TYPE           0
QUANT_ADDITIONAL_CARDS                0
POSTAL_ADDRESS_TYPE                   0
SEX                                   0
MARITAL_STATUS                        0
QUANT_DEPENDANTS                      0
EDUCATION_LEVEL                       0
STATE_OF_BIRTH                        0
CITY_OF_BIRTH                         0
NACIONALITY                           0
RESIDENCIAL_STATE                     0
RESIDENCIAL_CITY                      0
RESIDENCIAL_BOROUGH                   0
FLAG_RESIDENCIAL_PHONE                0
RESIDENCIAL_PHONE_AREA_CODE           0
RESIDENCE_TYPE                     1349
MONTHS_IN_RESIDENCE                3777
FLAG_MOBILE_PHONE                     0
FLAG_EMAIL                            0
PERSONAL_MONTHLY_INCOME               0
OTHER_INCOMES                         0
FLAG_VISA                             0
FLAG_MASTERCARD                       0


In [22]:
# Let's drop columns with sum of NaN values greater than the half of the total
# number of samples
cols_to_drop = list(data.isna().sum()[
    data.isna().sum() > data.shape[0]//2].index)
cols_to_drop

['PROFESSIONAL_CITY',
 'PROFESSIONAL_BOROUGH',
 'MATE_PROFESSION_CODE',
 'EDUCATION_LEVEL_2']

In [23]:
# Append columns with just one unique value to dols_to drop as they don't add
# value to the analysis
for col in data.columns:
    if len(data[col].unique()) == 1:
        if col not in cols_to_drop:
            cols_to_drop.append(col)

In [24]:
print('The columns to drop before correlation analysis are', cols_to_drop)

The columns to drop before correlation analysis are ['PROFESSIONAL_CITY', 'PROFESSIONAL_BOROUGH', 'MATE_PROFESSION_CODE', 'EDUCATION_LEVEL_2', 'CLERK_TYPE', 'QUANT_ADDITIONAL_CARDS', 'EDUCATION_LEVEL', 'FLAG_MOBILE_PHONE', 'FLAG_HOME_ADDRESS_DOCUMENT', 'FLAG_RG', 'FLAG_CPF', 'FLAG_INCOME_PROOF', 'FLAG_ACSP_RECORD']


In [25]:
# Drop columns with non-relevant information
train_data = data.drop(columns=cols_to_drop)

In [26]:
# Take a look on data inside the training dataset
train_data.head(10)

,PAYMENT_DAY,APPLICATION_SUBMISSION_TYPE,POSTAL_ADDRESS_TYPE,SEX,MARITAL_STATUS,QUANT_DEPENDANTS,STATE_OF_BIRTH,CITY_OF_BIRTH,NACIONALITY,RESIDENCIAL_STATE,...,FLAG_PROFESSIONAL_PHONE,PROFESSIONAL_PHONE_AREA_CODE,MONTHS_IN_THE_JOB,PROFESSION_CODE,OCCUPATION_TYPE,PRODUCT,AGE,RESIDENCIAL_ZIP_3,PROFESSIONAL_ZIP_3,TARGET_LABEL_BAD=1
0,,,,,,,,,,,,,,,,,,,,,
1,5,Web,1,F,6,1,RN,Assu,1,RN,...,N,,0,9.0,4.0,1,32,595,595,1
2,15,Carga,1,F,2,0,RJ,rio de janeiro,1,RJ,...,N,,0,11.0,4.0,1,34,230,230,1
3,5,Web,1,F,2,0,RN,GARANHUNS,1,RN,...,N,,0,11.0,NaN,1,27,591,591,0
4,20,Web,1,F,2,0,PE,CABO,1,PE,...,N,,0,NaN,NaN,1,61,545,545,0
5,10,Web,1,M,2,0,RJ,RIO DE JANEIRO,1,RJ,...,N,,0,9.0,5.0,1,48,235,235,1
6,10,0,1,M,2,0,MG,CARMO DO RIO CLARO,1,MG,...,N,,0,9.0,2.0,2,40,371,371,1
7,15,Carga,1,F,2,2,BA,salvador,1,BA,...,N,,0,11.0,4.0,1,40,413,413,1
8,25,Web,1,F,1,0,MG,ATALEIA,1,SP,...,Y,5,0,11.0,2.0,1,28,686,686,0
9,15,0,1,F,1,0,SP,LENCOIS PAULISTA,1,SP,...,N,,0,0.0,2.0,2,31,172,172,0


Before imputing missing values, It is necessary to contrast the kind of values we have in each variable and the expected values given by the file 'PAKDD2010_VariablesList.XLS'

In [27]:
# Build a dictionary with expected values for categorical variables
expected_values = {
    'PAYMENT_DAY': [1, 5, 10, 15, 20, 25],
    'APPLICATION_SUBMISSION_TYPE': ['Web', 'Carga'],
    'QUANT_ADDITIONAL_CARDS': [1, 2],
    'POSTAL_ADDRESS_TYPE': [1, 2],
    'SEX': ['M', 'F'],
    'MARITAL_STATUS': [1, 2, 3, 4, 5, 6, 7],
    'EDUCATION_LEVEL': [1, 2, 3, 4, 5],
    'NACIONALITY': [0, 1, 2],
    'FLAG_RESIDENTIAL_PHONE': ['Y', 'N'],
    'RESIDENCE_TYPE': [1, 2, 3, 4 ,5],
    'FLAG_MOBILE_PHONE': ['Y', 'N'],
    'FLAG_EMAIL': [0, 1],
    'FLAG_VISA': [0, 1],
    'FLAG_MASTERCARD': [0, 1],
    'FLAG_DINERS': [0, 1],
    'FLAG_AMERICAN_EXPRESS': [0, 1],
    'FLAG_OTHER_CARDS': [0, 1],
    'QUANT_BANKING_ACCOUNTS': [0, 1, 2],
    'QUANT_SPECIAL_BANKING_ACCOUNTS': [0, 1, 2],
    'COMPANY': ['Y', 'N'],
    'FLAG_PROFESSIONAL_PHONE': ['Y', 'N'],
    'OCCUPATION_TYPE': [1, 2, 3, 4, 5],
    'EDUCATION_LEVEL_2': [1, 2, 3, 4, 5],
    'FLAG_HOME_ADDRESS_DOCUMENT': [0, 1],
    'FLAG_RG': [0, 1],
    'FLAG_CPF': [0, 1],
    'FLAG_INCOME_PROOF': [0, 1],
    'FLAG_ACSP_RECORD': ['Y', 'N'],
    'TARGET_LABEL_BAD=1': [0, 1]
}

In [28]:
for col in train_data.columns:
    if col in expected_values:
        # Define a categorical data type
        cat_dtype = pd.CategoricalDtype(categories=expected_values[col],
                                        ordered=False)
        train_data[col] = train_data[col].astype(dtype=cat_dtype)

In [29]:
# Get the numerical variables
num_vars = train_data._get_numeric_data().columns
# Get the categorical variables
cat_vars = list(set(train_data.columns) - set(num_vars))

In [30]:
num_vars

Index(['QUANT_DEPENDANTS', 'MONTHS_IN_RESIDENCE', 'PERSONAL_MONTHLY_INCOME',
       'OTHER_INCOMES', 'PERSONAL_ASSETS_VALUE', 'QUANT_CARS',
       'MONTHS_IN_THE_JOB', 'PROFESSION_CODE', 'PRODUCT', 'AGE'],
      dtype='object')

In [31]:
cat_vars

['STATE_OF_BIRTH',
 'FLAG_VISA',
 'FLAG_AMERICAN_EXPRESS',
 'TARGET_LABEL_BAD=1',
 'RESIDENCIAL_BOROUGH',
 'QUANT_SPECIAL_BANKING_ACCOUNTS',
 'FLAG_DINERS',
 'PROFESSIONAL_PHONE_AREA_CODE',
 'FLAG_OTHER_CARDS',
 'APPLICATION_SUBMISSION_TYPE',
 'CITY_OF_BIRTH',
 'FLAG_PROFESSIONAL_PHONE',
 'RESIDENCIAL_ZIP_3',
 'MARITAL_STATUS',
 'PROFESSIONAL_STATE',
 'SEX',
 'COMPANY',
 'QUANT_BANKING_ACCOUNTS',
 'POSTAL_ADDRESS_TYPE',
 'FLAG_RESIDENCIAL_PHONE',
 'PROFESSIONAL_ZIP_3',
 'RESIDENCIAL_PHONE_AREA_CODE',
 'FLAG_MASTERCARD',
 'PAYMENT_DAY',
 'RESIDENCIAL_STATE',
 'FLAG_EMAIL',
 'RESIDENCIAL_CITY',
 'OCCUPATION_TYPE',
 'RESIDENCE_TYPE',
 'NACIONALITY']

Dataframe method '_get_numeric_data' returns variables whose data type is numeric but if we take a look on those columns, we can conclude that they are some variables that are categorical but expressed by numbers, so we have to find another way to split the data between numerical and categorical variables.

The new criteria will be the number of unique values that are present in the data for each column, so those variables that have less than 20 different values will be considered as categorical because 20 different values inside a dataset with 50000 samples is a small value so each of those 20 numbers will be repeated (in average) a lot of times.

In [32]:
'QUANT_DEPENDANTS' not in expected_values.keys()

True

In [33]:
'QUANT_DEPENDANTS' in num_vars

True

In [34]:
if train_data['QUANT_DEPENDANTS'].nunique() <= 20 and 'QUANT_DEPENDANTS' not in expected_values.keys():
    print('Ok')


Ok


In [35]:
# Let's cast numerical variables with less than 20 unique values to string type
for col in num_vars:
    if train_data[col].nunique() <= 20 and col in expected_values.keys():
        # First cast to int
        train_data[col] = pd.to_numeric(train_data[col],
                                        errors='coerce',     # Errors to NaN
                                        downcast='integer')
        # Create categorical data type for its unique values
        cat_dtype = pd.CategoricalDtype(
            categories=train_data[col].dropna().unique())
        # Cast to categorical data type
        train_data[col] = train_data[col].astype(dtype=cat_dtype)

Let's get the new numerical and categorical variables

In [36]:
# Get the numerical variables
num_vars = train_data._get_numeric_data().columns
# Get the categorical variables
cat_vars = list(set(train_data.columns) - set(num_vars))

In [37]:
num_vars

Index(['QUANT_DEPENDANTS', 'MONTHS_IN_RESIDENCE', 'PERSONAL_MONTHLY_INCOME',
       'OTHER_INCOMES', 'PERSONAL_ASSETS_VALUE', 'QUANT_CARS',
       'MONTHS_IN_THE_JOB', 'PROFESSION_CODE', 'PRODUCT', 'AGE'],
      dtype='object')

In [38]:
cat_vars

['STATE_OF_BIRTH',
 'FLAG_VISA',
 'FLAG_AMERICAN_EXPRESS',
 'TARGET_LABEL_BAD=1',
 'RESIDENCIAL_BOROUGH',
 'QUANT_SPECIAL_BANKING_ACCOUNTS',
 'FLAG_DINERS',
 'PROFESSIONAL_PHONE_AREA_CODE',
 'FLAG_OTHER_CARDS',
 'APPLICATION_SUBMISSION_TYPE',
 'CITY_OF_BIRTH',
 'FLAG_PROFESSIONAL_PHONE',
 'RESIDENCIAL_ZIP_3',
 'MARITAL_STATUS',
 'PROFESSIONAL_STATE',
 'SEX',
 'COMPANY',
 'QUANT_BANKING_ACCOUNTS',
 'POSTAL_ADDRESS_TYPE',
 'FLAG_RESIDENCIAL_PHONE',
 'PROFESSIONAL_ZIP_3',
 'RESIDENCIAL_PHONE_AREA_CODE',
 'FLAG_MASTERCARD',
 'PAYMENT_DAY',
 'RESIDENCIAL_STATE',
 'FLAG_EMAIL',
 'RESIDENCIAL_CITY',
 'OCCUPATION_TYPE',
 'RESIDENCE_TYPE',
 'NACIONALITY']

Now, it will be good to take a look on how many different values do the categorical variables have, in order to know how many variables will appear when we apply One Hot Encoding to those variables with a high number of different values.

In [39]:
train_data[cat_vars].nunique()

STATE_OF_BIRTH                       29
FLAG_VISA                             2
FLAG_AMERICAN_EXPRESS                 2
TARGET_LABEL_BAD=1                    2
RESIDENCIAL_BOROUGH               14511
QUANT_SPECIAL_BANKING_ACCOUNTS        3
FLAG_DINERS                           2
PROFESSIONAL_PHONE_AREA_CODE         87
FLAG_OTHER_CARDS                      2
APPLICATION_SUBMISSION_TYPE           2
CITY_OF_BIRTH                      9910
FLAG_PROFESSIONAL_PHONE               2
RESIDENCIAL_ZIP_3                  1481
MARITAL_STATUS                        7
PROFESSIONAL_STATE                   28
SEX                                   2
COMPANY                               2
QUANT_BANKING_ACCOUNTS                3
POSTAL_ADDRESS_TYPE                   2
FLAG_RESIDENCIAL_PHONE                2
PROFESSIONAL_ZIP_3                 1481
RESIDENCIAL_PHONE_AREA_CODE         102
FLAG_MASTERCARD                       2
PAYMENT_DAY                           6
RESIDENCIAL_STATE                    27


As they are variables with a high number of different values, we will drop them to keep a dataset with a shape we can handle after One Hot Encoding.

In [40]:
cols_to_drop = []
# List columns to drop
for col in cat_vars:
    if train_data[col].nunique() > 80:
        cols_to_drop.append(col)

# Drop columns
train_data.drop(columns=cols_to_drop, inplace=True)

In [41]:
# Take a look on the remaining dataframe
train_data.head(10)

,PAYMENT_DAY,APPLICATION_SUBMISSION_TYPE,POSTAL_ADDRESS_TYPE,SEX,MARITAL_STATUS,QUANT_DEPENDANTS,STATE_OF_BIRTH,NACIONALITY,RESIDENCIAL_STATE,FLAG_RESIDENCIAL_PHONE,...,QUANT_CARS,COMPANY,PROFESSIONAL_STATE,FLAG_PROFESSIONAL_PHONE,MONTHS_IN_THE_JOB,PROFESSION_CODE,OCCUPATION_TYPE,PRODUCT,AGE,TARGET_LABEL_BAD=1
0,,,,,,,,,,,,,,,,,,,,,
1,5,Web,1,F,6,1,RN,1,RN,Y,...,0,N,,N,0,9.0,4,1,32,1
2,15,Carga,1,F,2,0,RJ,1,RJ,Y,...,0,Y,,N,0,11.0,4,1,34,1
3,5,Web,1,F,2,0,RN,1,RN,Y,...,0,N,,N,0,11.0,NaN,1,27,0
4,20,Web,1,F,2,0,PE,1,PE,N,...,0,N,,N,0,NaN,NaN,1,61,0
5,10,Web,1,M,2,0,RJ,1,RJ,Y,...,0,N,,N,0,9.0,5,1,48,1
6,10,NaN,1,M,2,0,MG,1,MG,Y,...,1,Y,MG,N,0,9.0,2,2,40,1
7,15,Carga,1,F,2,2,BA,1,BA,Y,...,0,N,,N,0,11.0,4,1,40,1
8,25,Web,1,F,1,0,MG,1,SP,N,...,0,Y,SP,Y,0,11.0,2,1,28,0
9,15,NaN,1,F,1,0,SP,1,SP,Y,...,1,Y,,N,0,0.0,2,2,31,0


As we can see, there are son samples that have a string like ' ' and this is just a missing value, so let's replace them with NaN so that pandas and numpy can recognize them as missing values.

In [42]:
# Count empty strings present in each column
train_data.isin(values=[' ']).sum()

PAYMENT_DAY                           0
APPLICATION_SUBMISSION_TYPE           0
POSTAL_ADDRESS_TYPE                   0
SEX                                   0
MARITAL_STATUS                        0
QUANT_DEPENDANTS                      0
STATE_OF_BIRTH                     2064
NACIONALITY                           0
RESIDENCIAL_STATE                     0
FLAG_RESIDENCIAL_PHONE                0
RESIDENCE_TYPE                        0
MONTHS_IN_RESIDENCE                   0
FLAG_EMAIL                            0
PERSONAL_MONTHLY_INCOME               0
OTHER_INCOMES                         0
FLAG_VISA                             0
FLAG_MASTERCARD                       0
FLAG_DINERS                           0
FLAG_AMERICAN_EXPRESS                 0
FLAG_OTHER_CARDS                      0
QUANT_BANKING_ACCOUNTS                0
QUANT_SPECIAL_BANKING_ACCOUNTS        0
PERSONAL_ASSETS_VALUE                 0
QUANT_CARS                            0
COMPANY                               0


In [43]:
# Replace empty strings
train_data.replace(to_replace=['', ' '], value=np.nan, inplace=True)

Finally, we'll save the dataset into a csv file.

In [44]:
train_data.to_csv('./preprocessed_data.csv')